<a href="https://colab.research.google.com/github/gunisha30/codes/blob/master/Copy_of_audio2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install praat-parselmouth

In [0]:
import librosa as l
import os
import numpy as np
os.chdir('/content/drive/My Drive/awarathon')
arr,sr=l.load('George.wav')
arr.size#/sr #length
#print(sr)
2871072/22050
u=22049*5
l=u-22049
arrnew=arr[l:u+1]
np.sum(arrnew)/22050

In [0]:
#convert mp4 to wav 
import sys
import os
from moviepy.editor import *

# name is the path, if its in the root directory then its just the video name
def convert_to_audio(name):
  video = VideoFileClip(name)
  audio = video.audio
  audio.write_audiofile('darshanq6.wav')

#folder renaming: first-Nishkarsh second-Yash third-Vishesh

os.chdir('/content/drive/My Drive/awarathon/videos')
name="darshanq6.mp4"
convert_to_audio(name)

In [0]:
import parselmouth
from parselmouth.praat import call, run_file
import inspect as i
import sys
import scipy.io.wavfile
import math
import moviepy.editor as mp


def mysppaus(m,p,l):              #pause rate and articulation rate
    sound=p+"/"+m+".wav"          #entire path till audio file
    sourcerun="/content/drive/My Drive/awarathon/myspsolution.praat"                  #path till mysolution.praat
    path=p                        #path till the folder which has audio file
    try:
        objects= run_file(sourcerun, -20, 2, 1, "yes",sound,path, 80, 400, 0.01, capture_output=True) 
        z1=str(objects[1])
        z2=z1.strip().split()
        print(z2)
        print ("articulation rate=",z2[3])
        p=int(z2[1])                          #pause length in secs
        #p=p/60                                #pause length in mins
        a=p/l                                 #ratio of pause length and total length
        print("rate of pause=",a)
        print ("pause percentage=",a*100)
        #print(p)
    except:
        print ("Try again the sound of the audio was not clear")
    return;


audioname="darshanq6"
path1="/content/drive/My Drive/awarathon/videos"

#length of video files 

clip = mp.VideoFileClip('/content/drive/My Drive/awarathon/videos/darshanq6.mp4')
duration = math.floor(clip.duration)
image_time_interval = 2
#duration=duration/60
print("length=",duration)

mysppaus(audioname,path1,duration)

length= 29
['A', 'noisy', 'background', 'or', 'unnatural-sounding', 'speech', 'detected.', 'No', 'result', 'try', 'again']
articulation rate= or
Try again the sound of the audio was not clear


In [0]:
import scipy.io.wavfile
import os
os.chdir('/content/drive/My Drive/awarathon')
rate, arr = scipy.io.wavfile.read("George.wav")
print(arr.shape[0])
print(arr.size)
length=arr.shape[0]/rate
print(length/60)
  


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!pip install youtube_dl

In [0]:
import youtube_dl

def download_from_vimeo(vid_link):
    ydl = youtube_dl.YoutubeDL({'outtmpl':'vid.mp4'})

    with ydl:
        result = ydl.extract_info(
            vid_link,
            download=True # We just want to extract the info
        )

    if 'entries' in result:
        # Can be a playlist or a list of videos
        video = result['entries'][0]
    else:
        # Just a video
        video = result

    print(video)

download_from_vimeo('https://player.vimeo.com/video/348632880')